## Imports

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from pandas.api.types import is_numeric_dtype

In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


## Dataset Import

In [ ]:
raw_h1b_df = pd.read_csv('./drive/Shareddrives/550/datasets/raw_data/h1b_data.csv')

In [ ]:
raw_company_reviews_df = pd.read_csv('./drive/Shareddrives/550/datasets/raw_data/company_reviews.csv')

In [ ]:
raw_h1b_df.columns

In [ ]:
raw_company_reviews_df.columns

# EDA

### Drop unused Columns

Identify the columns that are not used for our purpose and can be dropped.

In [ ]:
company_drop = ['ceo_approval', 'ceo_count', 'roles', 'salary', 
                'interview_count', 'headquarters', 'revenue', 'website']

In [ ]:
h1b_drop = ['emp_zip', 'emp_country', 'soc_code', 'soc_name',
            'prevailing_wage', 'pw_unit', 'pw_level', 'wage_to',
            'wage_unit', 'work_city', 'work_state', 'emp_h1b_dependent',
            'emp_willful_violator', 'lat', 'lng']

In [ ]:
raw_h1b_filtered_df = raw_h1b_df.drop(h1b_drop, axis=1)

In [ ]:
raw_company_reviews_filtered_df = raw_company_reviews_df.drop(company_drop, axis=1)

### EDA helper functions

In [ ]:
# Analyze the NULL value percentage in dataset
def NullPercentage(df):
  print("Dataset NULL value percentage.")
  print(df.apply(lambda col : col.isnull().sum()/len(col)))
  print("\n")

In [ ]:
def CompanyNameCleanup(name):
  ret = name.lower().replace(", inc.", "").replace(",inc.", "").replace(", inc", "").replace(" inc.", "").replace(",inc", "").replace(" inc", "")
  ret = ret.replace(", llp.", "").replace(",llp.", "").replace(", llp", "").replace(" llp.", "").replace(",llp", "").replace(" llp", "")
  ret = ret.replace(", llc.", "").replace(",llc.", "").replace(", llc", "").replace(" llc.", "").replace(",llc", "").replace(" llc", "")
  return ret

### H1-B Database EDA

In [ ]:
NullPercentage(raw_h1b_filtered_df)

##### EDA analysis
As all fields in the H1-B database are important for the purpose of our study, we need to drop entries that have at least one fields being NULL

In [ ]:
h1b_df = raw_h1b_filtered_df.dropna()

In [ ]:
NullPercentage(h1b_df)

In [ ]:
h1b_df.dtypes

Size of the H1B data after dropping NULL values is:

In [ ]:
len(h1b_df)

##### Process and Cleanup String Values

**full_time_position** field

We will modify the `full_time_position` field to be boolean for easier query later

In [ ]:
h1b_df.loc[:, 'full_time_position'] = h1b_df['full_time_position'].apply(lambda x : True if x == 'Y' else False)

**emp_name** field

In [ ]:
h1b_df['emp_name'].unique

In [ ]:
h1b_df[h1b_df['emp_name'] == 'zionsville maria montessori international acad..']['emp_name'] = 'zionsville maria montessori international academy'

In [ ]:
h1b_df.loc[:, 'emp_name'] = h1b_df['emp_name'].apply(lambda x : CompanyNameCleanup(x))

In [ ]:
# h1b_filtered_df['emp_name'] = h1b_filtered_df['emp_name'].apply(lambda x : x.lower())
# h1b_filtered_df['emp_name'] = h1b_filtered_df['emp_name'].apply(lambda x : x.replace(", inc.", " inc"))
# h1b_filtered_df['emp_name'] = h1b_filtered_df['emp_name'].apply(lambda x : x.replace(", llp", " llp"))
# h1b_filtered_df['emp_name'] = h1b_filtered_df['emp_name'].apply(lambda x : x.replace(", llc", " llc"))

In [ ]:
h1b_df.info()

**emp_city** field

In [ ]:
h1b_df['emp_city'].unique

In [ ]:
h1b_df.loc[:, 'emp_city'] = h1b_df['emp_city'].apply(lambda x : x.lower())

**job_title** field

In [ ]:
h1b_df.loc[:, 'job_title'] = h1b_df['job_title'].apply(lambda x : str(x).lower())

In [ ]:
h1b_df.info()

### Company Reviews Database EDA

In [ ]:
NullPercentage(raw_company_reviews_filtered_df)

##### EDA analysis
We noticed that there are multiple fields having NULL values. Attributes such as `name`, `rating` are important values for our analysis, and entries with these fields being NULL values should be dropped. On the other hand, fields such as `reviews`(reviews breakdown), `description`(text reviews), interview related stats, `employees`(number of employees), and `industry` are good-to-have values that will improve how informative our analysis would be, and it is ok for them to be NULL for certain companies. 

In [ ]:
company_reviews_df = raw_company_reviews_filtered_df.dropna(subset=['name', 'rating'])

Size of company review data after dropping NULL value is: 

In [ ]:
len(company_reviews_df)

##### Process and Cleanup String Values

###### name

In [ ]:
company_reviews_df['name'].unique()

In [ ]:
company_reviews_df.loc[:, 'name'] = company_reviews_df['name'].apply(lambda x : CompanyNameCleanup(x))

In [ ]:
company_reviews_df['name'].unique()

##### industry

In [ ]:
company_reviews_df['industry'].unique()

In [ ]:
"Restaurants, Travel and Leisure\nRestaurants".replace('\n', '/')

In [ ]:
company_reviews_df.loc[:, 'industry'] = company_reviews_df['industry'].apply(lambda x : str(x).replace('\n', '/'))

In [ ]:
company_reviews_df['industry'].unique()

Size of dataframe after processing:

In [ ]:
len(company_reviews_df)

In [ ]:
company_reviews_df.info()

### Attempt to merge

In [ ]:
# merged_df = pd.merge(
#     h1b_df, company_reviews_df, left_on="emp_name", right_on='name', how="inner", suffixes=('_left', '_right')
# )

In [ ]:
# len(merged_df)

## Explode Nested Fields for Main Datasets

Company reviews dataset

In [ ]:
company_reviews_exploded_df = company_reviews_df.copy()

In [ ]:
company_reviews_df['locations']

In [ ]:
company_reviews_df[company_reviews_df['name'] == 'google']

In [ ]:
company_reviews_exploded_df.loc[:, 'locations'] = company_reviews_df['locations'].apply(lambda x : re.findall("\'([A-Za-z\\s]+),\\s([A-Z]{2})\':\\s", x))

In [ ]:
company_reviews_exploded_df = company_reviews_exploded_df.explode('locations')

In [ ]:
company_reviews_exploded_df['locations']

In [ ]:
re.findall("\(\'([A-Za-z\\s]+)\',\\s\'[A-Z]{2}\'\)", "('Scottsdale', 'AZ')")[0]

In [ ]:
company_reviews_exploded_df['city'] = company_reviews_exploded_df['locations'].apply(lambda x : re.findall("\(\'([A-Za-z\\s]+)\',\\s\'[A-Z]{2}\'\)", str(x)))

In [ ]:
company_reviews_exploded_df['city']

In [ ]:
company_reviews_exploded_df.loc[:, 'city'] = company_reviews_exploded_df['city'].apply(lambda x : x[0] if len(x) > 0 else np.NaN)

In [ ]:
company_reviews_exploded_df['state'] = company_reviews_exploded_df['locations'].apply(lambda x : re.findall("\(\'[A-Za-z\\s]+\',\\s\'([A-Z]{2})\'\)", str(x)))

In [ ]:
company_reviews_exploded_df.loc[:, 'state']  = company_reviews_exploded_df['state'].apply(lambda x : x[0] if len(x) > 0 else np.NaN)

In [ ]:
company_reviews_exploded_df = company_reviews_exploded_df.drop('locations', axis=1)

## Split Into Tables

In [ ]:
company_reviews_df.columns

#### Industry

In [ ]:
industry_df = pd.DataFrame({"industryId": range(0, len(company_reviews_df['industry'].unique())), 
                            "industry": company_reviews_df['industry'].unique()})

In [ ]:
industry_df = industry_df[industry_df['industry'] != 'nan']

In [ ]:
industry_df = industry_df.drop_duplicates().reset_index(drop=True)

In [ ]:
industry_df.dtypes

In [ ]:
industry_df.head(3)

#### Location

In [ ]:
h1b_df.columns

In [ ]:
h1b_df.loc[:, 'emp_city'] = h1b_df['emp_city'].apply(lambda x : x.split(",")[0])

In [ ]:
location_df = pd.DataFrame({ "city": h1b_df['emp_city'], 
                            "state": h1b_df['emp_state']})

In [ ]:
location_df = location_df.reset_index(drop=True)

In [ ]:
location_df = pd.concat([location_df, company_reviews_exploded_df[['city', 'state']]])

In [ ]:
location_df = location_df.drop_duplicates(subset=['city', 'state']).reset_index(drop=True)

In [ ]:
location_df = location_df.drop_duplicates(subset=['city', 'state']).reset_index(drop=True)

In [ ]:
location_df.insert(loc=0, column='locationId', value= range(0, len(location_df)))

In [ ]:
location_df.dtypes

#### Job

In [ ]:
job_df = pd.DataFrame({ "title": h1b_df['job_title'], 
                       "fulltime": h1b_df['full_time_position']})

In [ ]:
job_df = job_df.drop_duplicates(subset=['title', 'fulltime']).reset_index(drop=True)

In [ ]:
job_df.insert(loc=0, column='jobId', value= range(0, len(job_df)))

In [ ]:
job_df.dtypes

#### Company

In [ ]:
company_df = pd.DataFrame({ "name": company_reviews_exploded_df['name'], 
                            "employeeSize": company_reviews_exploded_df['employees'],
                            "industry": company_reviews_exploded_df['industry']})

In [ ]:
temp_df = h1b_df[['emp_name']].copy()

In [ ]:
temp_df = temp_df.rename(columns={'emp_name': 'name'}).dropna()

In [ ]:
company_df = pd.concat([company_df, temp_df]).drop_duplicates(subset=['name']).reset_index(drop=True)

In [ ]:
company_df.insert(loc=0, column='companyId', value= range(0, len(company_df)))

In [ ]:
industry_dict = {}
for index, row in industry_df.iterrows():
  industry_dict[row['industry']] = row['industryId']

In [ ]:
company_df['industryId'] = company_df['industry'].apply(lambda x : industry_dict[str(x)] if str(x) != 'nan' else np.nan)

In [ ]:
company_df['industryId'] = pd.Series(company_df['industryId']).astype('Int64')

In [ ]:
company_df['name'] = company_df['name'].astype('string') 

In [ ]:
company_df['employeeSize'] = company_df['employeeSize'].astype('string') 

In [ ]:
company_df['industry'] = company_df['industry'].astype('string') 

In [ ]:
company_df

#### Reviews

In [ ]:
company_dict = {}
for index, row in company_df[['name', 'companyId']].iterrows():
  company_dict[row['name']] = row['companyId']

In [ ]:
reviews_df = pd.DataFrame({ "name": company_reviews_exploded_df['name'], 
                            "overallRating": company_reviews_exploded_df['rating'],
                            "textReview": company_reviews_exploded_df['description'],
                            "workLifeBalance": company_reviews_exploded_df['ratings'].apply(lambda x : re.findall("\'Work/Life Balance\':\\s\'([0-9\.]+)'", str(x))),
                            "compensationOrBenefits": company_reviews_exploded_df['ratings'].apply(lambda x : re.findall("\'Compensation/Benefits\':\\s\'([0-9\.]+)'", str(x))),
                            "jobSecurityOrAdvance": company_reviews_exploded_df['ratings'].apply(lambda x : re.findall("\'Job Security/Advancement\':\\s\'([0-9\.]+)'", str(x))),
                            "management": company_reviews_exploded_df['ratings'].apply(lambda x : re.findall("\'Management\':\\s\'([0-9\.]+)'", str(x))),
                            "culture": company_reviews_exploded_df['ratings'].apply(lambda x : re.findall("\'Culture\':\\s\'([0-9\.]+)'", str(x))),
                            "happiness": company_reviews_exploded_df['happiness'].apply(lambda x : re.findall("\'Work Happiness Score\':\\s\'([0-9\.]+)'", str(x))), 
                            "numReviews": company_reviews_exploded_df['reviews'].apply(lambda x : re.findall("([0-9]+)\\sreviews", str(x)))})

In [ ]:
ratings_cols = ['workLifeBalance', 'compensationOrBenefits', 'jobSecurityOrAdvance', 'management', 'culture', 'happiness', 'numReviews']
for col in ratings_cols:
  reviews_df.loc[:, col] = reviews_df[col].apply(lambda x : x[0] if len(x) > 0 else np.NaN)

In [ ]:
reviews_df['companyId'] = reviews_df['name'].apply(lambda x : company_dict[str(x)] if str(x) != 'nan' else np.nan)

In [ ]:
reviews_df['companyId'] = pd.Series(reviews_df['companyId']).astype('Int64')

In [ ]:
reviews_df = reviews_df.drop(columns=['name'])

In [ ]:
reviews_df.insert(loc=0, column='reviewId', value= range(0, len(reviews_df)))

In [ ]:
reviews_df.head(3)

In [ ]:
reviews_df.dtypes

#### InterviewReview

In [ ]:
def mapInterviewDurationToTime(duration_str):
  if duration_str == 'About a day or two':
    return 2
  elif duration_str == 'About a week':
    return 7
  elif duration_str == 'About two weeks':
    return 14
  elif duration_str == 'About a month':
    return 10
  elif duration_str == 'More than one month':
    return 60
  else:
    return np.NaN

In [ ]:
company_reviews_exploded_df.loc[:, 'interview_duration'] = company_reviews_exploded_df['interview_duration'].apply(lambda x : mapInterviewDurationToTime(x))

In [ ]:
interviewReview_df = pd.DataFrame({ "name": company_reviews_exploded_df['name'], 
                                    "timeline(days)": company_reviews_exploded_df['interview_duration'],
                                    "experience": company_reviews_exploded_df['interview_experience'],
                                    "difficulty": company_reviews_exploded_df['interview_difficulty']})

In [ ]:
interviewReview_df['companyId'] = interviewReview_df['name'].apply(lambda x : company_dict[str(x)] if str(x) != 'nan' else np.nan)

In [ ]:
interviewReview_df['companyId'] = pd.Series(interviewReview_df['companyId']).astype('Int64')

In [ ]:
interviewReview_df = interviewReview_df.drop(columns=['name'])

In [ ]:
interviewReview_df.insert(loc=0, column='interviewReviewId', value= range(0, len(interviewReview_df)))

In [ ]:
interviewReview_df.dtypes

#### H1BCase

In [ ]:
job_df

In [ ]:
job_dict = {}
for index, row in job_df.iterrows():
  job_dict[(row['title'], row['fulltime'])] = row['jobId']

In [ ]:
h1b_df

In [ ]:
h1bCase_df = pd.DataFrame({ "emp_name": h1b_df['emp_name'], 
                            "job_title": h1b_df['job_title'],
                           'fulltime': h1b_df['full_time_position'],
                            "caseStatus": h1b_df['case_status'],
                            "caseYear": h1b_df['case_year'], 
                            "submitDate": h1b_df['case_submitted'],
                            "decisionDate": h1b_df['decision_date'],
                            "wageFrom": h1b_df['wage_from'],
                           })

In [ ]:
h1bCase_df['companyId'] = h1bCase_df['emp_name'].apply(lambda x : company_dict[str(x)] if str(x) != 'nan' else np.nan)

In [ ]:
h1bCase_df['companyId'] = pd.Series(h1bCase_df['companyId']).astype('Int64')

In [ ]:
h1bCase_df['jobId'] = h1bCase_df.apply(lambda x : (job_dict[(x['job_title'], x['fulltime'])]), axis=1)

In [ ]:
h1bCase_df.insert(loc=0, column='h1bCaseId', value= range(0, len(h1bCase_df)))

In [ ]:
h1bCase_df.dtypes

#### HasRole

In [ ]:
hasRole_df = pd.DataFrame({"company": h1b_df['emp_name'], 
                           "city": h1b_df['emp_city'],
                            "state": h1b_df['emp_state'],
                            "job_title": h1b_df['job_title'], 
                            "fulltime": h1b_df['full_time_position']})

In [ ]:
location_df

In [ ]:
location_dict = {}
for index, row in location_df.iterrows():
  location_dict[(row['city'], row['state'])] = row['locationId']

In [ ]:
hasRole_df['locationId'] = hasRole_df.apply(lambda x : (location_dict[(x['city'], x['state'])]), axis=1)

In [ ]:
hasRole_df['jobId'] = hasRole_df.apply(lambda x : (job_dict[(x['job_title'], x['fulltime'])]), axis=1)

In [ ]:
hasRole_df['companyId'] = hasRole_df['company'].apply(lambda x : company_dict[x])

In [ ]:
hasRole_df = hasRole_df.drop(columns=['company', 'city', 'state', 'job_title', 'fulltime'])

In [ ]:
hasRole_df = hasRole_df.drop_duplicates(subset=['locationId', 'jobId', 'companyId']).reset_index(drop=True)

In [ ]:
hasRole_df.insert(loc=0, column='hasRoleId', value= range(0, len(hasRole_df)))

In [ ]:
hasRole_df.dtypes

#### InterestedIn (not populated until have users...?)

# Write Processed Dataset to Drive

**Uncomment and run ONLY IF you want to write to drive**


In [ ]:
# industry_df.to_csv('./drive/Shareddrives/550/datasets/updated_datasets/industry_df.csv')

In [ ]:
# location_df.to_csv('./drive/Shareddrives/550/datasets/updated_datasets/location_df.csv')

In [ ]:
# job_df.to_csv('./drive/Shareddrives/550/datasets/updated_datasets/job_df.csv')

In [ ]:
# company_df.to_csv('./drive/Shareddrives/550/datasets/updated_datasets/company_df.csv')

In [ ]:
# reviews_df.to_csv('./drive/Shareddrives/550/datasets/updated_datasets/reviews_df.csv')

In [ ]:
# interviewReview_df.to_csv('./drive/Shareddrives/550/datasets/updated_datasets/interviewReview_df.csv')

In [ ]:
# h1bCase_df.to_csv('./drive/Shareddrives/550/datasets/updated_datasets/h1bCase_df.csv')

In [ ]:
# hasRole_df.to_csv('./drive/Shareddrives/550/datasets/updated_datasets/hasRole_df.csv')

#TODO: 
- Further string clean (to lower case, remove punctuation etc, and decide if it is necessary to calculate string similarity)

# Content below are outdated and needs to be modified

## Simple EDA

In [ ]:
def SimpleNumericAnalysis(df):
  numeric_df = df.select_dtypes(include=[np.number])
  print("Mean values analysis")
  print(numeric_df.apply(lambda col : col.mean()))
  print("\n")
  print("Standard deviation analysis")
  print(numeric_df.apply(lambda col : col.std()))
  print("\n")

In [ ]:
def SimpleDbAnalysis(df):
  print("Dataset column types:\n", df.dtypes)
  print("\n")
  SimpleNumericAnalysis(df)
  print("Dataset size: ", len(df.index))